In [1]:
import csv
import gensim

from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models

en_stop = get_stop_words('en')
p_stemmer = PorterStemmer()

# run lda on words themselves
# https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html
# whoops. didn't work
def read_docs(filepath):
    docs = []
    with open(filepath, "rb") as csvfile:
        reader = csv.reader(csvfile, delimiter=",")
        for row in reader:
            docs.append([row[0],row[1]])
    return docs

doc_set = read_docs("data/cleaned/regular_verbs_clean.csv")
doc_set += read_docs("data/cleaned/irregular_verbs_clean.csv")

num_docs = len(doc_set)
texts = []
for i in range(num_docs):
    # remove stop words from tokens
    # stem token
    texts.append([p_stemmer.stem(t) for t in doc_set[i] if not t in en_stop])

dictionary = corpora.Dictionary(texts)
# corpus is a document-term matrix.  a list of vectors equal to the number of documents. 
# In each document vector is a series of tuples
corpus = [dictionary.doc2bow(text) for text in texts]
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20)
print(ldamodel.print_topics(num_topics=10, num_words=3))

[(0, u'0.014*beat + 0.014*bust + 0.014*devel'), (1, u'0.013*dibbl + 0.013*dray + 0.013*alkyl'), (2, u'0.012*dream + 0.012*divert + 0.012*fever'), (3, u'0.015*quit + 0.015*honi + 0.015*shred'), (4, u'0.012*bit + 0.012*leav + 0.012*inpour'), (5, u'0.013*saw + 0.013*menac + 0.013*wood'), (6, u'0.013*bled + 0.013*raven + 0.013*dint'), (7, u'0.026*let + 0.019*sing + 0.013*legat'), (8, u'0.013*emulsifi + 0.013*reenter + 0.013*slime'), (9, u'0.013*bereav + 0.013*stereograph + 0.013*overag')]


In [2]:
import csv
import string
import sys, itertools
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score

N = 1000
alphabet = string.ascii_lowercase + '#' + '.'
D = len(alphabet)
z = np.ones(N)

RI_pres = np.random.rand(D, N)
RI_pres = np.where(RI_pres>0.5, 1, -1)

RI_past = np.random.rand(D, N)
RI_past = np.where(RI_past>0.5, 1, -1)

def ngram_encode(ngram_str, letter_vecs, window=3):
    vec = np.zeros(letter_vecs.shape[1])
    full_str = '#' + ngram_str + '.'
    for il, l in enumerate(full_str[:-(window-1)]):
        trivec = letter_vecs[alphabet.find(full_str[il]), :]
        for c3 in range(1, window):
            trivec = trivec * np.roll(letter_vecs[alphabet.find(full_str[il+c3]), :], c3)
        vec += trivec
    return vec

def ngram_encode_cl(ngram_str, letter_vecs, window=3):
    vec = np.zeros(letter_vecs.shape[1])
    full_str = '#' + ngram_str + '.'
    for il, l in enumerate(full_str[:-(window-1)]):
        trivec = letter_vecs[alphabet.find(full_str[il]), :]
        for c3 in range(1, window):
            trivec = trivec * np.roll(letter_vecs[alphabet.find(full_str[il+c3]), :], c3)
        vec += trivec
    return 2* (vec + 0.1*(np.random.rand(letter_vecs.shape[1])-0.5) < 0) - 1



def read_csv(filepath):
    # lists of lists where each sublist corresponds to rule
    pres, past = {}, {}
    key = 0
    num_words = 0
    with open(filepath, "rb") as csvfile:
        reader = csv.reader(csvfile, delimiter=",")
        sub_pres, sub_past = [], []
        for row in reader:
            if "#" in row[0]:
                pres[key], past[key] = sub_pres, sub_past
                sub_pres, sub_past = [], []
                key += 1
            else:
                sub_pres.append(ngram_encode_cl(row[0], RI_pres, 3))
                sub_past.append(ngram_encode_cl(row[1], RI_past, 3))
                num_words += 2
                
    X = np.zeros((num_words-4, N))
    y = np.zeros(num_words-4)
    num_classes = len(pres.keys())

    j = 0
    for i in range(num_classes):
        for pres_vec in pres[i]:
            X[j] = pres_vec
            y[j] = i
            j += 1
        for past_vec in past[i]:
            X[j] = past_vec
            y[j] = i
            j += 1
        
        #print i
    return X, y, num_classes

def plot_proj_2D(X, y, num_classes):
    d = 2
    U, s, V = np.linalg.svd(X, full_matrices=True)
    print (U.shape, s.shape, V.shape) 

    Ud = U[:, :d]
    Sd = s[:d]*np.eye(d)
    Vd = V[:d, :]
    SdVd = np.dot(Sd, Vd)
    print Ud.shape, Sd.shape, Vd.shape, SdVd.shape
    reduced_X = Ud
        
    colors = cm.rainbow(np.linspace(0, 1, num_classes))
    print colors
    plt.figure()
    plt.scatter(reduced_X[:,0], reduced_X[:,1], color = colors[y.astype(int)])
    plt.show()
    """

    colors = cm.rainbow(np.linspace(0, 1, num_classes))

    fig = plt.figure()
    for i, row in data.iterrows():
        plt.scatter(proj[0], proj[1], color = 
                    plotlabels[row[self.labelcol]])
    plt.show()
    """
    
"""
http://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html

irregular verbs are in own classes separated by first letter
regular verbs are in own classes separated by rules: #d, #ed, #**ed, #, #?
"""
regular_X, regular_y, regular_num_classes = read_csv("data/cleaned/regular_verbs_clean_categorize_small.csv")
irregular_X, irregular_y, irregular_num_classes = read_csv("data/cleaned/irregular_verbs_final_categorize.csv")

In [5]:
print (regular_X.shape, regular_y.shape)
regular_clf = LinearDiscriminantAnalysis(n_components=4, store_covariance=True)
regular_clf.fit(regular_X, regular_y)
print(regular_clf.predict(np.random.random(N)))
#test_indices = np.random.choice(regular_X.shape[0], 20, replace=False)
regular_y_pred = regular_clf.predict(regular_X)#regular_X[test_indices])
print regular_y_pred
print accuracy_score(regular_y, regular_y_pred)

((104, 1000), (104,))
[ 1.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
0.903846153846


/Users/quinntran/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [6]:
# so arbitrarily assigned rules for irregular verbs don't work lmao. 
# should every irregular verb just get its own rule?
irregular_clf = LinearDiscriminantAnalysis(n_components=9, store_covariance=True)
irregular_clf.fit(irregular_X, irregular_y)
print(irregular_clf.predict(np.random.random(N)))
irregular_y_pred = regular_clf.predict(irregular_X)
print irregular_y
print irregular_y_pred
print accuracy_score(irregular_y, irregular_y_pred)

[ 2.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  2.  2.  2.  2.  2.  2.  2.  2.
  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
  2.  2.  2.  2.  2.  2.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.
  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.
  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  5.  5.  5.  5.  5.  5.
  5.  5.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  7.  7.  7.  7.
  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.
  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1

/Users/quinntran/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [7]:
d = 2
U, s, V = np.linalg.svd(regular_X, full_matrices=True)
print (U.shape, s.shape, V.shape) 

Ud = U[:, :d]
Sd = s[:d]*np.eye(d)
Vd = V[:d, :]
SdVd = np.dot(Sd, Vd)
print Ud.shape, Sd.shape, Vd.shape, SdVd.shape
reduced_X = Ud

((104, 104), (104,), (1000, 1000))
(104, 2) (2, 2) (2, 1000) (2, 1000)


In [ ]:
# why is graphing so slow!?
plt.figure()
plt.scatter(reduced_X[:,0], reduced_X[:,1])
plt.show()